In [40]:
import pandas as pd
import numpy as np

s = '/Users/morganwaddington/Documents/Tandon/Classes/Machine-Learning/kddcup.data_10_percent_corrected'
df = pd.read_csv(s, header=None, encoding='utf-8')

df = df.sample(frac=0.2, random_state=1) 

In [41]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in [1, 2, 3]:
    df.iloc[:, col] = label_encoder.fit_transform(df.iloc[:, col])


In [42]:

print(df.iloc[0, 41])

y = df.iloc[:, 41].values 

y = np.where(np.isin(y, ['back.', 'land.', 'neptune.', 'pod.', 'smurf.', 'teardrop.']), 0, 1) 

X = df.iloc[:, 0:41].values #41 is correct


smurf.


In [43]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)


In [44]:

print('Labels counts in y:', np.bincount(y))
print('Labels counts in y_train:', np.bincount(y_train))
print('Labels counts in y_test:', np.bincount(y_test))


Labels counts in y: [78346 20458]
Labels counts in y_train: [62677 16366]
Labels counts in y_test: [15669  4092]


In [45]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

from imblearn.under_sampling import RandomUnderSampler
under = RandomUnderSampler(random_state=1)
X_train_std, y_train = under.fit_resample(X_train_std, y_train)


In [46]:
print('Labels counts in y_train:', np.bincount(y_train_std))

Labels counts in y_train: [16366 16366]


In [47]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
def plot_decision_regions(X, y, classifier, test_idx=None,
                             resolution=0.02):
       # setup marker generator and color map
       markers = ('s', 'x', 'o', '^', 'v')
       colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
       cmap = ListedColormap(colors[:len(np.unique(y))])
       # plot the decision surface
       x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
       x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
       xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                              np.arange(x2_min, x2_max, resolution))
       Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
       Z = Z.reshape(xx1.shape)
       plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
       plt.xlim(xx1.min(), xx1.max())
       plt.ylim(xx2.min(), xx2.max())
       for idx, cl in enumerate(np.unique(y)):
           plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                       alpha=0.8, c=colors[idx],
                       marker=markers[idx], label=cl,
                       edgecolor='black')

        # highlight test examples
       if test_idx:
           # plot all examples
           X_test, y_test = X[test_idx, :], y[test_idx]
           plt.scatter(X_test[:, 0], X_test[:, 1],
                       c='', edgecolor='black', alpha=1.0,
                       linewidth=1, marker='o',
                       s=100, label='test set')

In [48]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))


In [71]:

from sklearn.svm import SVC

svm_linear = SVC(kernel='linear', C=1.0, random_state=1)
svm_linear.fit(X_train_std, y_train)

rbf = SVC(kernel='rbf', C=1.0, gamma=0.1, random_state=1)
rbf.fit(X_train_std, y_train)

sigmoid = SVC(kernel='sigmoid', C=1.0, gamma=0.001, random_state=1)
sigmoid.fit(X_train_std, y_train)

poly = SVC(kernel='poly', C=1.0, gamma=0.1, random_state=1)
poly.fit(X_train_std, y_train)



# plot_decision_regions(X_combined_std,
#                       y_combined,
#                       classifier=svm,
#                       test_idx=X_test_std)

# plt.xlabel('petal length [standardized]')
# plt.ylabel('petal width [standardized]')
# plt.legend(loc='upper left')
# plt.tight_layout()
# plt.show()




# from sklearn.linear_model import LogisticRegression

# lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs')
# lr.fit(X_train_std, y_train);


# print(f"Converged in {lr.n_iter_[0]} iterations.")

SVC(gamma=0.1, kernel='poly', random_state=1)

Although I'm not an expert, it appears that the model converging in 61 iterations is quite standard and nothing unusual. This number of iterations may indicate some moderate complexity in the data, which isn't surprising given the extensive nature of the dataset. Ultimately, the model successfully converges and exhibits high accuracy, so this outcome seems well within normal expectations.

In [72]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from IPython.display import display, HTML

def print_results(y_train_pred, y_test_pred):   
    cm = confusion_matrix(y_test, y_test_pred)
    print('Confusion Matrix:')
    print(cm,'\n')
    
    
    report = classification_report(y_test, y_test_pred)
    print('Classification Report:')
    print(report)

y_train_pred = svm_linear.predict(X_train_std)
y_test_pred = svm_linear.predict(X_test_std)
display(HTML(f"<h2>Test result for LINEAR Kernel:</h2>"))
print_results(y_train_pred, y_test_pred)

y_train_pred = rbf.predict(X_train_std)
y_test_pred = rbf.predict(X_test_std)
display(HTML(f"<h2>Test result for RBF Kernel:</h2>"))
print_results(y_train_pred, y_test_pred)

y_train_pred = sigmoid.predict(X_train_std)
y_test_pred = sigmoid.predict(X_test_std)
display(HTML(f"<h2>Test result for SIGMOID Kernel:</h2>"))
print_results(y_train_pred, y_test_pred)

y_train_pred = poly.predict(X_train_std)
y_test_pred = poly.predict(X_test_std)
display(HTML(f"<h2>Test result for POLY Kernel:</h2>"))
print_results(y_train_pred, y_test_pred)


Confusion Matrix:
[[15667     2]
 [    3  4089]] 

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15669
           1       1.00      1.00      1.00      4092

    accuracy                           1.00     19761
   macro avg       1.00      1.00      1.00     19761
weighted avg       1.00      1.00      1.00     19761



Confusion Matrix:
[[15662     7]
 [    0  4092]] 

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15669
           1       1.00      1.00      1.00      4092

    accuracy                           1.00     19761
   macro avg       1.00      1.00      1.00     19761
weighted avg       1.00      1.00      1.00     19761



Confusion Matrix:
[[15577    92]
 [    7  4085]] 

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     15669
           1       0.98      1.00      0.99      4092

    accuracy                           0.99     19761
   macro avg       0.99      1.00      0.99     19761
weighted avg       1.00      0.99      1.00     19761



Confusion Matrix:
[[15667     2]
 [    1  4091]] 

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15669
           1       1.00      1.00      1.00      4092

    accuracy                           1.00     19761
   macro avg       1.00      1.00      1.00     19761
weighted avg       1.00      1.00      1.00     19761

